In [34]:
import picos
import numpy as np

In [89]:
n = 4
alpha = 0.001
pi = np.array([0.25, 0.5, 1/8, 1/8])
S = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if i != j:
            S[i,j] = i+j
            

model = picos.Problem()
P = picos.RealVariable('P', (n,n))
X = picos.RealVariable('X', (n,n))
W = picos.Constant('W', S)
v1 = picos.Constant('1_n', np.ones(n))
q = picos.Constant('q', np.sqrt(pi))
Pi = picos.Constant('pi', pi)


PI_sqrt = picos.Constant('PI^.5', np.diag(np.sqrt(pi)))
PI_negsqrt = picos.Constant('PI^-.5', np.diag(1/np.sqrt(pi)) )
T = np.eye(n) - PI_sqrt*P*PI_negsqrt + q*q.T
print(T)
M = picos.block([[T, "I"],["I", X]])
print(M)
model.set_objective('min', picos.min(alpha*picos.trace(X) +(1-alpha)*Pi.T*(P^W)*v1))

#and now some constraints
model.add_constraint(M.hermitianized >> 0)
model.add_list_of_constraints([ sum(P[i,:]) == 1 for i in range(n)])
model.add_list_of_constraints([ pi[i]*P[i,j]==pi[j]*P[j,i] for i in range(n) for j in range(i+1, n)])
model.add_list_of_constraints([ P[i,j]<=1 for i in range(n) for j in range(n) if i!=j])
model.add_list_of_constraints([ 0<=P[i,j] for i in range(n) for j in range(n) if i!=j])
model.add_list_of_constraints([P[i,i]==0 for i in range(n)])
print(model)

[4×4] - PI^.5·P·PI^-.5 + q·qᵀ
[[4×4] - PI^.5·P·PI^-.5 + q·qᵀ, I; I, X]
Semidefinite Program
  minimize 0.001·tr(X) + 0.999·piᵀ·(P⊙W)·1_n
  over
    4×4 real variable P, X
  subject to
    ([[4×4] - PI^.5·P·PI^-.5 + q·qᵀ, I; I, X] + [[4×4] -
      PI^.5·P·PI^-.5 + q·qᵀ, I; I, X]ᵀ)/2 ≽ 0
    P[i,:][0] + P[i,:][1] + P[i,:][2] + P[i,:][3] = 1 ∀ i ∈ [0…3]
    0.i·P[j,k] = 0.l·P[k,j] ∀ (i,j,k,l) ∈
      zip([25,25,…,5,125],[0,0,…,1,2],[1,2,…,3,3],[5,125,…,125,125])
    P[i,j] ≤ 1 ∀ (i,j) ∈ zip([0,0,…,3,3],[1,2,…,1,2])
    P[i,j] ≥ 0 ∀ (i,j) ∈ zip([0,0,…,3,3],[1,2,…,1,2])
    P[i,i] = 0 ∀ i ∈ [0…3]


In [90]:
model.solve()

<primal feasible solution pair (claimed optimal) from cvxopt>

In [91]:
print(P)

[-7.08e-15  1.00e+00 -4.71e-12 -4.69e-12]
[ 5.00e-01  2.67e-14  2.50e-01  2.50e-01]
[-9.41e-12  1.00e+00 -4.53e-15  1.96e-11]
[-9.41e-12  1.00e+00  1.96e-11 -5.33e-15]


In [74]:
Pnp = np.array(P.value)

In [79]:
Pnp[Pnp<1e-10]=0


In [80]:
Pnp

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 0.00000000e+00, 2.50000000e-01, 2.50000000e-01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 3.84010385e-10],
       [0.00000000e+00, 1.00000000e+00, 3.78657627e-10, 0.00000000e+00]])

In [81]:
for i in range(n):
    Pnp[i,:]/=np.sum(Pnp[i,:])

In [82]:
Pnp

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 0.00000000e+00, 2.50000000e-01, 2.50000000e-01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 3.84010385e-10],
       [0.00000000e+00, 1.00000000e+00, 3.78657627e-10, 0.00000000e+00]])